In [1]:
# Yernar Shambayev, DL-2
# Используя сверточные слои и архитектуру получите на fashion mnist качество не ниже 89,5%

import torch
from torchvision import datasets, transforms
from torch import nn, optim
import torch.nn.functional as F

In [2]:
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize([0.], [0.5])])

trainset = datasets.FashionMNIST('.', download = True, train = True, transform = transform)
testset = datasets.FashionMNIST('.', download = True, train = False, transform = transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size = 64, shuffle = True)
testloader = torch.utils.data.DataLoader(testset, batch_size = 64, shuffle = True)

In [3]:
dataiter = iter(trainloader)
images, labels = dataiter.next()
print(images.shape, labels.shape)

torch.Size([64, 1, 28, 28]) torch.Size([64])


In [4]:
class convolutionalNet(nn.Module):
    def __init__(self):
        super(convolutionalNet, self).__init__()

        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=12, kernel_size=5)

        self.fc1 = nn.Linear(in_features=12*4*4, out_features=120)
        self.fc2 = nn.Linear(in_features=120, out_features=60)
        self.out = nn.Linear(in_features=60, out_features=10)

    def forward(self, t):
        t = self.conv1(t)
        t = F.relu(t)
        t = F.max_pool2d(t, kernel_size=2, stride=2)

        t = self.conv2(t)
        t = F.relu(t)
        t = F.max_pool2d(t, kernel_size=2, stride=2)

        t = t.reshape(-1, 12 * 4 * 4)
        t = self.fc1(t)
        t = F.relu(t)

        t = self.fc2(t)
        t = F.relu(t)

        t = self.out(t)
        return t

In [5]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
model = convolutionalNet()
model.to(device)

cuda:0


convolutionalNet(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 12, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=192, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=60, bias=True)
  (out): Linear(in_features=60, out_features=10, bias=True)
)

In [6]:
loss = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.002)
epochs = 20

for epoch in range(epochs):
    model.train()
    for fashion_images, fashion_labels in trainloader:
        fashion_images, fashion_labels = fashion_images.to(device), fashion_labels.to(device)

        optimizer.zero_grad()
        output = model(fashion_images)
        l = loss(output, fashion_labels)
        l.backward()
        optimizer.step()

    test_loss = 0
    accuracy = 0
    with torch.no_grad():
        model.eval()

        for fashion_images, fashion_labels in testloader:
            fashion_images, fashion_labels = fashion_images.to(device), fashion_labels.to(device)
            test_guess = model(fashion_images)
            test_loss += loss(test_guess, fashion_labels)

            prob = torch.exp(test_guess)
            top_probs, top_classes = prob.topk(1, dim=1)
            equals = fashion_labels == top_classes.view(fashion_labels.shape)
            accuracy += torch.mean(equals.type(torch.FloatTensor))

    print(f'Эпоха: {epoch + 1}/{epochs}. Потери: {test_loss / len(testloader):.4f}. \
    Точность: {accuracy / len(testloader):.4f}')

Эпоха: 1/20. Потери: 0.4656.     Точность: 0.8348
Эпоха: 2/20. Потери: 0.3954.     Точность: 0.8543
Эпоха: 3/20. Потери: 0.3477.     Точность: 0.8714
Эпоха: 4/20. Потери: 0.3331.     Точность: 0.8782
Эпоха: 5/20. Потери: 0.3265.     Точность: 0.8832
Эпоха: 6/20. Потери: 0.3123.     Точность: 0.8867
Эпоха: 7/20. Потери: 0.3247.     Точность: 0.8809
Эпоха: 8/20. Потери: 0.3263.     Точность: 0.8844
Эпоха: 9/20. Потери: 0.3001.     Точность: 0.8911
Эпоха: 10/20. Потери: 0.3143.     Точность: 0.8839
Эпоха: 11/20. Потери: 0.3148.     Точность: 0.8902
Эпоха: 12/20. Потери: 0.3055.     Точность: 0.8931
Эпоха: 13/20. Потери: 0.3168.     Точность: 0.8896
Эпоха: 14/20. Потери: 0.3204.     Точность: 0.8920
Эпоха: 15/20. Потери: 0.3199.     Точность: 0.8891
Эпоха: 16/20. Потери: 0.3278.     Точность: 0.8929
Эпоха: 17/20. Потери: 0.3053.     Точность: 0.8993
Эпоха: 18/20. Потери: 0.3227.     Точность: 0.8941
Эпоха: 19/20. Потери: 0.3443.     Точность: 0.8926
Эпоха: 20/20. Потери: 0.3436.     Точнос